In [ ]:
!pip3 -q install nltk
!pip3 -q install wordcloud

# Import

In [1]:
import os
import datetime

import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud

import matplotlib as mpl
from konlpy.tag import Twitter
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [2]:
# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

font_dirs = ['../static/NanumBarunGothic.ttf']
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)

sns.set(font_scale=1.2)
# set font
plt.rcParams['font.family'] = 'NanumBarunGothic'

# Args

입력 받아야 하는 변수들

- date_week
- instagram_follower
- youtube_subscriners
- facebook_followers

# Load Datas

In [3]:
date_week = "2021-11-24"

FOLLOWERS = dict()
FOLLOWERS['instragram'] = 2426
FOLLOWERS['youtube'] = 4350
FOLLOWERS['facebook'] = 576

In [4]:
data_path = f"../sns-data/{date_week}"
webnames = ['youtube', 'facebook', 'article', 'datalab']

In [5]:
from pathlib import Path

def mkdir_folders(date_week, webnames):
    p = Path(f"../sns-data/{date_week}")
    p.mkdir(parents=True, exist_ok=True)
    
    Path(f"../weekly_plots/{date_week}").mkdir(parents=True, exist_ok=True)
    
    for webname in webnames:
        (p / webname).mkdir(parents=True, exist_ok=True)

In [6]:
mkdir_folders(date_week, webnames)

In [295]:
def load_dataframes(data_path, s_date, e_date):
    datas = dict()
    
    datas['youtube_table'] = pd.read_csv(f"{data_path}/youtube/Table data.csv")
    
    # youtube follower
    datas['youtube_follower'] = pd.read_csv(f"{data_path}/youtube/Totals.csv", parse_dates=['Date'])
    datas['youtube_follower'] = datas['youtube_follower'][(datas['youtube_follower']['Date'] >= s_date) &
                             (datas['youtube_follower']['Date'] <= e_date)]
    
    s_follower = FOLLOWERS['youtube'] - datas['youtube_follower']['Subscribers'].sum()
    datas['youtube_follower']['Subscribers'] = datas['youtube_follower']['Subscribers'].cumsum() + s_follower
    datas['youtube_follower'].columns = ['Date', 'youtube_Counts']
    datas['youtube_follower']['youtube_Texts'] = datas['youtube_follower']["youtube_Counts"].astype(str)
    
    # datas['youtube_follower']['sns_type'] = 'Youtube'

    # facebook follower
    datas['facebook_follower'] = pd.read_csv(f"{data_path}/facebook/facebook_insights.csv",
                                        usecols=['날짜', 'Daily New Likes'],
                                        parse_dates=['날짜'])
    datas['facebook_follower'].drop(0, inplace=True)
    datas['facebook_follower'].columns = ['Date', 'facebook_Counts']
    datas['facebook_follower'] = datas['facebook_follower'][(datas['facebook_follower']['Date'] >= s_date) &
                             (datas['facebook_follower']['Date'] <= e_date)]
    # datas['facebook_follower']['sns_type'] = "Facebook"
    datas['facebook_follower'] = datas['facebook_follower'].fillna(0)
    datas['facebook_follower']['facebook_Counts'] = datas['facebook_follower']["facebook_Counts"].map(int)
    
    total_sum = datas['facebook_follower']['facebook_Counts'].sum()
    facebook_follower_start = FOLLOWERS['facebook'] - total_sum
    datas['facebook_follower']['facebook_Counts'] = datas['facebook_follower']['facebook_Counts'].cumsum() + facebook_follower_start
    datas['facebook_follower']['facebook_Texts'] = datas['facebook_follower']["facebook_Counts"].astype(str)
    
    # instagram followers
    datas['instagram_follower'] = datas['facebook_follower'].loc[:, 'Date':'Date'].copy()
    datas['instagram_follower']["instagram_Counts"] = np.linspace(1518, 1889, len(datas['instagram_follower'])).astype(int)
    datas['instagram_follower']['instagram_Texts'] = datas['instagram_follower']["instagram_Counts"].astype(str)
    
    # article
    datas['article_org'] = pd.read_csv(f"{data_path}/article/article.csv", parse_dates=['pubDate'],
                                      usecols=['title', 'pubDate'])
    datas['article'] = pd.read_csv(f"{data_path}/article/article.csv", parse_dates=['pubDate'],
                           usecols=['title', 'pubDate'])
    datas['article'] = datas['article'][['title', 'pubDate']].groupby('pubDate').count().reset_index()
    datas['article'].columns = ['Date', 'article_Counts']
    datas['article']['article_Texts'] = datas['article']["article_Counts"].astype(str)
    
    datas['datalab'] = pd.read_csv("../sns-data/2021-11-24/datalab/datalab.csv", skiprows=7, names=['Date', "버즈량"])
    
    return datas

In [296]:
from datetime import datetime, timedelta

e_date = datetime.strptime(date_week, '%Y-%m-%d')
s_date = e_date - timedelta(days=7)

In [297]:
datas = load_dataframes(data_path, s_date, e_date)

In [10]:
left_df = pd.DataFrame([], columns=['Date'])

for right_name in ['youtube_follower', 'facebook_follower', 'instagram_follower', 'article']:
    right_df = datas[right_name]
    left_df = pd.merge(left=left_df, right=right_df, on="Date", how='outer')

In [11]:
left_df

,Date,youtube_Counts,youtube_Texts,facebook_Counts,facebook_Texts,instagram_Counts,instagram_Texts,article_Counts,article_Texts
0,2021-11-17,4141.0,4141,554,554,1518,1518,NaN,NaN
1,2021-11-18,4181.0,4181,555,555,1579,1579,9.0,9
2,2021-11-19,4229.0,4229,557,557,1641,1641,7.0,7
3,2021-11-20,4285.0,4285,561,561,1703,1703,5.0,5
4,2021-11-21,4322.0,4322,566,566,1765,1765,21.0,21
5,2021-11-22,4350.0,4350,569,569,1827,1827,54.0,54
6,2021-11-23,NaN,NaN,576,576,1889,1889,61.0,61


In [12]:
left_df.iloc[:, ::2] = left_df.iloc[:, ::2].fillna('')

In [13]:
datas.keys()

dict_keys(['youtube_table', 'youtube_follower', 'facebook_follower', 'instagram_follower', 'article', 'datalab'])

In [67]:
from bokeh.models import ColumnDataSource

source = ColumnDataSource(left_df)
source2 = ColumnDataSource(left_df.copy())

In [85]:
source2.data

{'index': array([0, 1, 2, 3, 4, 5, 6]),
 'Date': array(['2021-11-17T00:00:00.000000000', '2021-11-18T00:00:00.000000000',
        '2021-11-19T00:00:00.000000000', '2021-11-20T00:00:00.000000000',
        '2021-11-21T00:00:00.000000000', '2021-11-22T00:00:00.000000000',
        '2021-11-23T00:00:00.000000000'], dtype='datetime64[ns]'),
 'youtube_Counts': array([4141., 4181., 4229., 4285., 4322., 4350.,   nan]),
 'youtube_Texts': array(['4141', '4181', '4229', '4285', '4322', '4350', ''], dtype=object),
 'facebook_Counts': array([554, 555, 557, 561, 566, 569, 576]),
 'facebook_Texts': array(['554', '555', '557', '561', '566', '569', '576'], dtype=object),
 'instagram_Counts': array([1518, 1579, 1641, 1703, 1765, 1827, 1889]),
 'instagram_Texts': array(['1518', '1579', '1641', '1703', '1765', '1827', '1889'],
       dtype=object),
 'article_Counts': array([nan,  9.,  7.,  5., 21., 54., 61.]),
 'article_Texts': array(['', '9', '7', '5', '21', '54', '61'], dtype=object)}

In [144]:
from bokeh.layouts import gridplot, column, row
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d, DateRangeSlider, CustomJS
from bokeh.models import CDSView, IndexFilter, BooleanFilter
from bokeh.plotting import figure, output_notebook, show

output_notebook()

def draw_lineplot(source, title, width, height, color="#1b1b1b", by='instagram'):
    assert by in ['instagram', 'facebook', 'youtube', 'article']
    
    date_key = 'Date'
    count_key = by + '_Counts'
    text_key = by + '_Texts'
    
#     booleans = [False if np.isnan(y_val) else True for y_val in source.data[count_key]]
#     view = CDSView(source=source, filters=[BooleanFilter(booleans)])
    
    p = figure(width=width, height=height, x_axis_type='datetime')
    
    p.title.text = title
    p.title.vertical_align = 'top'
    p.title.text_line_height = 0.1

    p.title.align = 'center'
    p.title.text_color = color
    p.title.text_font_size = "15px"
    p.title_location = 'above'
    
    p.grid.grid_line_color = 'white'
    p.xaxis.axis_line_color = color
    p.yaxis.axis_line_color = color
    
    lineplot = p.line(date_key, count_key, source=source, line_width=2, color=color, alpha=0.4)
    circle_plot = p.circle(date_key, count_key, source=source, fill_color='white', size=8)

    labels = LabelSet(x=date_key, y=count_key, text=text_key,
                  x_offset=0, y_offset=10, source=source, render_mode='css')
    
    p.add_layout(labels)
    return p

Loading BokehJS ...

In [174]:
callback = CustomJS(args=dict(source=source, ref_source=source2), code="""
    const date_from = Date.parse(new Date(cb_obj.value[0]).toISOString());
    const date_to = Date.parse(new Date(cb_obj.value[1]).toISOString());
    
    const data = source.data;
    const ref = ref_source.data;
    
    const from_pos = ref["Date"].indexOf(date_from);
    const to_pos = ref["Date"].indexOf(date_to) + 1;
    
    const names = ['youtube', 'facebook', 'instagram', 'article'];
    
    for (let i=0; i<names.length; i++){
        let name = names[i];
        let count_name = name + '_Counts';
        let text_name = name + '_Texts';
        
        data[count_name] = ref[count_name].slice(from_pos, to_pos);
        data[text_name] = ref[text_name].slice(from_pos, to_pos);
    }

    source.change.emit();
    """)

In [146]:
real_e_date = e_date - timedelta(days=1)

In [147]:
d_range.start

datetime.datetime(2021, 11, 17, 0, 0)

In [153]:
from bokeh.models import Title

In [155]:
t = Title()

In [175]:
t.text = 'aa'

In [310]:
output_file("sns_dashboard.html")

total_width = 1000
total_height = 600

fig_width = int(total_width // 2)
fig_height = int(total_height // 2)

p1 = draw_lineplot(source, '페이스북', width=fig_width, height=fig_height, color="#3b5998", by='facebook')
p2 = draw_lineplot(source, '인스타그램', width=fig_width, height=fig_height, color="#FF7F00", by='instagram')
p3 = draw_lineplot(source, '유튜브', width=fig_width, height=fig_height, color='#c4302b', by='youtube')
p4 = draw_lineplot(source, '기사', width=fig_width, height=fig_height, color='#000080', by='article')

day_step = 24 * 60 * 60 * 1000
date_slider = DateRangeSlider(value=(s_date, real_e_date), start=s_date, end=real_e_date, step=day_step,
                              title="조회 기간",
                             format="%Y/%m/%d")

p2.x_range = p1.x_range
p3.x_range = p1.x_range
p4.x_range = p1.x_range

# date_range = DataRange1d
d_range = DataRange1d(start=s_date, end=real_e_date)

figures = [p1, p2, p3, p4]
date_slider.js_on_change('value', callback)
show(column(date_slider, row(p1, p2), row(p3, p4)))

In [206]:
temp_df = pd.read_csv('../sns-data/2021-11-24/youtube/Chart data view.csv', parse_dates=['Date', 'Video publish time'])
temp_df = temp_df.dropna()

In [266]:
view_df = temp_df.groupby("Date").sum().reset_index()
view_df['Texts'] = view_df.Views.astype(str)

title_df = temp_df.groupby(["Video title", "Video"]).sum().sort_values("Views", ascending=False).reset_index()
title_df['Texts'] = title_df.Views.astype(str)

In [267]:
title_df['y'] = title_df['Video title'].apply(lambda x: x[:17] + '...' if len(x) > 20 else x)

In [285]:
view_source = ColumnDataSource(view_df)
title_source = ColumnDataSource(title_df)

In [284]:
from bokeh.models import TapTool, OpenURL
    
def draw_popular_videos(title_source):
    youtube_url = 'https://www.youtube.com/watch?v=@Video'

    TOOLTIPS = [
        ("index", "$index"),
        ("title", "@{Video title}"),
        ("youtube", "Click Me!!")
    ]

    p = figure(width=800, height=400, title="인기 동영상", y_range=title_source.data['y'], tooltips=TOOLTIPS)

    p.add_tools(TapTool())
    p.hbar(y='y', right='Views', color='firebrick', height=0.5, left=0, source=title_source)

    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=youtube_url)

    return p

def draw_youtube_hits(view_source):
    p = figure(width=400, height=400, title="유튜브 조회수 추이", x_axis_type="datetime")
    p.vbar(x='Date', top='Views', bottom=0, color='firebrick', line_width=20, source=view_source)

    labels = LabelSet(x='Date', y='Views', text='Texts',
                      x_offset=-17, y_offset=5, render_mode='css', source=view_source,
                     text_font_size="10pt")

    p.add_layout(labels)
    return p

In [307]:
from bokeh.io import output_file

output_file("youtube_dashboard.html")

p5 = draw_youtube_hits(view_source)
p6 = draw_popular_videos(title_source)

show(row(p5, p6))

In [306]:
datas['article_org']['title'].apply(lambda x: x.replace("<b>김동연</b>", ""))

0              , YS 6주기에 &quot;YS의 기득권 깨기 정신 되새겨야&quot;
1                   &quot;진정한 모빌리티는 교통약자·소셜·리더십에서&quot;
2                 &quot;종부세·양도세 개편하겠다…실수요자 부담 최소화&quot;
3                      &quot;여야 선대위, 기득권의 바통 주고받기&quot;
4                        “1가구 1주택자 종부세·양도세 완화”…세제 공약 발표
                             ...                       
152    김영삼 전 대통령 6주기 추모식 자리한 이재명 - 윤석열 - 심상정 - 안철수 -...
153                YS 6주기 추모식 참석한 이재명-윤석열-심상정-안철수- 대선후보
154    5자대결…윤석열 47.7% 이재명 33.3% 심상정 4.7% 안철수 3.7%  1.8%
155       [머니S포토] YS 6주기 추모식, 추보발언하는 이재명·윤석열·심상정·안철수...
156                     [포토] 심상정·안철수·은 양당 체제 종식 공동선언하라!
Name: title, Length: 157, dtype: object

In [301]:
grs = datas['article_org'].groupby("pubDate")

In [303]:
grp_keys = list(grs.groups.keys())

In [302]:
for idx, group 

- 디자인 유무
- 인스타그램 Follower 수랑
- 대시보드 구성 (페이지 하나로?)
- 중복 그래프